In [19]:
exec(open('file_reader.py').read())

generating features..

 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 544294893146091520

 768859780240773121

 764927075522260992

 544274544174071809

 498486826269548545

 544315472075042818

 524923293711998976

 769988636754505729

 524936793633083394

 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 5249417202

In [20]:
df.head()


,classification,context_path,has_context,text,topic,opinion
580319078155468800,true,NaN,False,Germanwings Airbus A320 crashes in French Alps...,true,0
580319184652890113,false,NaN,False,BREAKING: 148 passengers were on board #German...,false,0
580320684305416192,unverified,NaN,False,Accident aircraft looks to be Germanwings (Air...,unverified,0
580321156508577792,false,NaN,False,Now hearing 148 passengers + crew on board the...,false,0
580322453928431617,false,NaN,False,German Wings airline tweeting now about report...,false,0


In [21]:
import sys
sys.path.insert(1, "./feature-extraction/twitter-features")
from VulgarExtractor import VulgarExtractor

In [22]:
wordlist = VulgarExtractor.vulgarWords("badwords.txt") 
dftext = df[['text']]
result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
df['isVulgar'] = result
df.head()

,classification,context_path,has_context,text,topic,opinion,isVulgar
580319078155468800,true,NaN,False,Germanwings Airbus A320 crashes in French Alps...,true,0,False
580319184652890113,false,NaN,False,BREAKING: 148 passengers were on board #German...,false,0,False
580320684305416192,unverified,NaN,False,Accident aircraft looks to be Germanwings (Air...,unverified,0,False
580321156508577792,false,NaN,False,Now hearing 148 passengers + crew on board the...,false,0,False
580322453928431617,false,NaN,False,German Wings airline tweeting now about report...,false,0,False


In [54]:
import TwitterParser
textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
tagged_sents = TwitterParser.tag(textlist)
print(len(tagged_sents))
df['POS'] = tagged_sents
df.head()

25


,classification,context_path,has_context,text,topic,opinion,isVulgar,POS
580319078155468800,true,NaN,False,Germanwings Airbus A320 crashes in French Alps...,true,0,False,"[(Germanwings, ^), (Airbus, ^), (A320, ^), (cr..."
580319184652890113,false,NaN,False,BREAKING: 148 passengers were on board #German...,false,0,False,"[(BREAKING, N), (:, ,), (148, $), (passengers,..."
580320684305416192,unverified,NaN,False,Accident aircraft looks to be Germanwings (Air...,unverified,0,False,"[(Accident, N), (aircraft, N), (looks, V), (to..."
580321156508577792,false,NaN,False,Now hearing 148 passengers + crew on board the...,false,0,False,"[(Now, R), (hearing, V), (148, $), (passengers..."
580322453928431617,false,NaN,False,German Wings airline tweeting now about report...,false,0,False,"[(German, A), (Wings, N), (airline, N), (tweet..."


In [62]:
word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

df['wordCount'] = word_counts
df['containsAdjective'] = contains_adjs
df['containsURL'] = contains_urls
df['containsEmoji'] = contains_emojis
df['containsAbbreviation'] = contains_abbrevs
df.head()

,classification,context_path,has_context,text,topic,opinion,isVulgar,POS,containsAdjective,wordCount,containsURL,containsEmoji,containsAbbreviation
580319078155468800,true,NaN,False,Germanwings Airbus A320 crashes in French Alps...,true,0,False,"[(Germanwings, ^), (Airbus, ^), (A320, ^), (cr...",False,9,True,False,False
580319184652890113,false,NaN,False,BREAKING: 148 passengers were on board #German...,false,0,False,"[(BREAKING, N), (:, ,), (148, $), (passengers,...",True,16,True,False,False
580320684305416192,unverified,NaN,False,Accident aircraft looks to be Germanwings (Air...,unverified,0,False,"[(Accident, N), (aircraft, N), (looks, V), (to...",False,16,True,False,False
580321156508577792,false,NaN,False,Now hearing 148 passengers + crew on board the...,false,0,False,"[(Now, R), (hearing, V), (148, $), (passengers...",True,18,False,False,False
580322453928431617,false,NaN,False,German Wings airline tweeting now about report...,false,0,False,"[(German, A), (Wings, N), (airline, N), (tweet...",True,17,True,False,False
